In [1]:
import os
import re
import nltk
import pickle
import contractions
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem  import WordNetLemmatizer


In [12]:
import pandas as pd

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')

[nltk_data] Downloading package stopwords to /home/anish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/anish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/anish/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/anish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /home/anish/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [4]:
filename = '../data/pickle_model.pkl'

In [5]:
with open(filename, 'rb') as file:  
    model = pickle.load(file)

print('File opened')

File opened


In [16]:
lemmatizer = WordNetLemmatizer()
stop_word = list(set(stopwords.words('english')))
stop_word = list(filter(lambda x : x!=str('not'),stop_word))

In [17]:
def text_processing(text):
    
    #removing everything except letter and space
    temp = re.sub(r"[^A-Za-z ]+",r"",text)
    #lower case
    temp = temp.lower()
    #remove unnecessary spaces
    temp = " ".join(temp.split())
    #word tokenize
    temp = word_tokenize(temp)
    # remove stop words
    temp = [ item for item in temp if item not in stop_word]
    #lemmatize
    temp = [lemmatizer.lemmatize(item) for item in temp]
    
    return temp

In [18]:
def noun_phrase(token):
  '''To extract important grammatical token from text'''

  key = pos_tag(token)
  data=[]
  for tag in key:
    if tag[1] == 'NN':
      data.append(tag[0])
  
  if len(data) == 0:
    return token
  
  return list(data)

In [61]:
travel=['heatuda','hetauda','dang','chitwan','gulmi','valley','jwalakhel','kathmandu','chitwan','manakamana','kurintar','dhading']

In [62]:
festival = ['janai','purnima','teej','rishi','panchami','dashain','tihar','tika','shravan','raksha', 'bandhan','rakchya']

In [63]:
ceremony = ['bratabandha','janti','shraddha','pooja','sorha','sraddha','pasni','shrada','puja']

In [79]:
def find_token(text):
    '''
    This function convert text, i.e string, into token
    after doing all required pre-processing

    argument: text -> input string to find token
    returns the token

    input: text of type string
    output: strings in list
    
    '''
    preprocess_text = text_processing(text)
    noun_phrase_token = noun_phrase(preprocess_text)
    for idx in range(len(noun_phrase_token)):
        if noun_phrase_token[idx] in travel:
            noun_phrase_token[idx] = 'travel'
        elif noun_phrase_token[idx] in festival:
            noun_phrase_token[idx] = 'festival'
        elif noun_phrase_token[idx] in ceremony:
            noun_phrase_token[idx] = 'ceremony'
            

    return noun_phrase_token

In [80]:
data_df = pd.read_csv('../data/leave_application.csv')

In [82]:
data_df['processed'] = data_df['reason'].map(find_token)